In [22]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()

#embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
new_db = FAISS.load_local('faiss_index', embeddings,  "db_from_texts_PP", allow_dangerous_deserialization=True)
    
    # if clients_goods:
    #     docs = await new_db.asimilarity_search_with_score(clients_goods, k=top_similar_documents)
    # else:
    #     docs = []

    # docs = [doc for doc, score in docs if score > trash_hold]
#docs_direct = new_db.similarity_search('расскажи про Пономареву Людмилу Викторовну', k=7)
#docs_direct = new_db.similarity_search('расскажи про парагрипп', include_metadata=True, k=7)
docs_direct = new_db.similarity_search('расскажи про Пономареву', include_metadata=True, k=7)
#docs_direct = new_db.similarity_search_with_relevance_scores('расскажи про парагрипп', k=7, score_threshold=0.3,)
# docs_direct = [doc for doc, score in docs_direct if score > 0]
docs=docs_direct
for doc in docs:
    print(f"{doc}\n")

# for doc, similarity_score in docs_direct:
#     print(f"Document: {doc.page_content}, Similarity Score: {similarity_score}")
#     print("-----------")

page_content='Пономарева Людмила Викторовна, филиал Волгоградская, 44 - хирург, невролог со стажем работы более 14 лет:  
Виды хирургических операций выполняемых Пономаревой Людмилой Викторовной:  
1. Овариогистерэктомия кошки, собаки
2. Кастрация кота, кобеля
3. Кастрация крипторха
4. Удаление пиометры
5. Удаление инородного тела
6. Оперативное лечение заворота желудка
7. Торакотомия (удаление доли легкого)
8. Холецистэктомия - удаление желчного пузыря
9. Энуклиация глаза - удаление глазного яблока
10. Хирургическая обработка/ушивание ран
11. Удаление промежностной, паховой, пупочной грыж
12. Сиалоэктомия - удаление слюнной железы
13. Цистотомия - удаление камней из мочевого пузыря
14. Уретростомия - операция по созданию постоянного искусственного наружного свища мочеиспускательного канала
15. Кесарево сечение у собак и кошек
16. Установка эзофагостомы
17. Латерализация хрящей гортани  
Решение о возможности/необходимости операции, расчет ее стоимости происходит на хирургическом прием

In [ ]:
    async def generate(self, documents: list[Document], ids: list):
        if self.db is None:
            self.db = FAISS.from_documents(
                documents=documents, embedding=self.embeddings, ids=ids
            )
        else:
            try:
                self.db.delete(ids)
                self.db.add_documents(documents=documents, ids=ids)
            except Exception as e:
                self.db.add_documents(documents=documents, ids=ids)

        self.db.save_local(
            folder_path=self.emb_db_directory, index_name=self.emb_db_name
        )

    async def generate_embeddings(self):
        print("Generating all embeddings...")

        async with db_helper.session_factory() as session:
            products = await db_get_products(session=session)

        documents = []
        ids = []

        for product in products:
            documents.append(
                Document(
                    page_content=product.full_name.lower(),
                    metadata={"name": product.full_name},
                )
            )
            ids.append(product.id)

        await self.generate(documents=documents, ids=ids)

In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()

#embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
new_db = FAISS.load_local('faiss_index', embeddings,  "db_from_texts_PP", allow_dangerous_deserialization=True)
# Получаем вектор запроса
query_vector = embeddings.embed_query('расскажи про парагрипп')

# Проверяем размерности
index_dimension = new_db.index.d
query_dimension = len(query_vector)

print(f"Размерность векторов в индексе: {index_dimension}")
print(f"Размерность вектора запроса: {query_dimension}")

Размерность векторов в индексе: 3072
Размерность вектора запроса: 3072


In [ ]:
from sentence_transformers import SentenceTransformer

# Загружаем модель для преобразования текста в вектор
model = SentenceTransformer('all-MiniLM-L6-v2')

# Текст, который нужно проверить
query_text = "Это текст, который нужно проверить."

# Преобразуем текст в вектор
query_vector = model.encode(query_text)